In [ ]:
# standard libraries
import numpy as np
import math
from pprint import pprint as pp
from matplotlib.pyplot import *
%matplotlib inline

In [ ]:
# user library
from bayesee.detector.spotter import Spotter
from bayesee.imaging.filter import *
from bayesee.imaging.image import *
from bayesee.operation.nb2d import *
from bayesee.operation.dp import *
from bayesee.operation.mathfunc import *

In [ ]:
# stimulus parameters
row = col = 128
ppd = 60
cpd = 1.5
cpi_v = row * cpd / ppd

In [ ]:
# simulation parameters
n_bin_spat_sim = 3
n_bin_amp_sim = 3
n_bin_background = 500
n_background = n_bin_background * n_bin_amp_sim * n_bin_spat_sim
background_mean = 128
background_std = 128 * 0.204

In [ ]:
# templates
sine_template = hann_window(row, col, row/2) * sine_wave(row, col, (cpi_v,0))
sine_template /= nb2dot(sine_template, sine_template)

In [ ]:
# backgrounds
uni_images = np.zeros((row, col, n_background))
cm_images = np.zeros((row, col, n_background))

for i in range(n_background):
    uni_images[:,:,i] = power_noise(row, col, -1, background_mean, background_std)
    cm_images[:,:,i] = contrast_modulate_lr(power_noise(row, col, -1, background_mean, background_std), 4)

In [ ]:
# spotters
TM = Spotter(method='TM')

In [ ]:
# simulation function
def detection_1_f_noise(model, template, background, amp_init):
    amp = np.ones((n_background*2,)) * amp_init
    stimulus = np.zeros_like(amp)
    response = np.zeros_like(amp)
    spat_sim = np.zeros_like(amp)
    amp_sim = np.zeros_like(amp)
    
    for i in range(n_background):
        stimulus[2*i] = 0
        response[2*i] = model.give_response(background[:,:,i], template)
        stimulus[2*i+1] = 1
        response[2*i+1] = model.give_response(add_to_center(background[:,:,i], amp_init*template), template)
        spat_sim[2*i] = spat_sim[2*i+1] = spatial_similarity(background[:,:,i], template)
        amp_sim[2*i] = amp_sim[2*i+1] = amplitude_similarity(background[:,:,i], template)
            
    return amp, stimulus, response, spat_sim, amp_sim

In [ ]:
# simulation
templates = [sine_template]
backgrounds = [uni_images, cm_images]
models = [TM]
amp_init = 0.001

amp_all = np.zeros((len(templates), len(backgrounds), len(models), n_background * 2))
stimulus_all = np.zeros_like(amp_all)
response_all = np.zeros_like(amp_all)
amp_sim_all = np.zeros_like(amp_all)
spat_sim_all = np.zeros_like(amp_all)

for t, template in enumerate(templates):
    for b, background in enumerate(backgrounds):
        amp_all[t,b,0,:], stimulus_all[t,b,0,:], response_all[t,b,0,:], spat_sim_all[t,b,0,:], amp_sim_all[t,b,0,:] = detection_1_f_noise(TM, template, background, amp_init)
        
np.savez('TM-spat-sim-amp-sim-1-f-noise', amp_all=amp_all, stimulus_all=stimulus_all, response_all=response_all, spat_sim_all=spat_sim_all, amp_sim_all=amp_sim_all)

In [ ]:
# calculate dp
bin_all = np.zeros_like(amp_all)
dp_all = np.zeros_like(amp_all)
for t, template in enumerate(templates):
    for b, background in enumerate(backgrounds):
        for i in range(n_bin_spat_sim):
            for j in range(n_bin_amp_sim):
                #should/2d-binning
                bin_idx = bin_all
                bin_amp, bin_stimulus, bin_response = amp_all[t,b,0,bin_idx], stimulus_all[t,b,0,bin_idx], response_all[t,b,0,bin_idx]
                dp_all[t,b,0,bin_idx], _ = glm_cont(bin_amp, bin_stimulus, bin_response)

In [ ]:
# plot function
def plot_double_bin_dp(spat_sim_all, amp_sim_all, dp_all, pargs):
    n_templates, n_backgrounds, n_models = db_thresholds.shape
    n_rows, n_cols = pargs['n_rows'], pargs['n_cols']
    fig, axs = subplots(nrows=n_rows, ncols=n_cols, figsize=pargs['figsize'], constrained_layout=True)
    db_th_min = db_thresholds.min() - 0.1 * db_thresholds.ptp()
    db_th_max = db_thresholds.max() + 0.1 * db_thresholds.ptp()
    
    for i in range(n_models):
        idxes = i//n_rows, i%n_rows
        axs[idxes].scatter(pargs['x_tick_labels'], db_thresholds[:,0,i], s=pargs['markersize'], marker=pargs['markers'][i], edgecolor=pargs['colors'][i], facecolor="none", linewidths=pargs['linewidth'], label=pargs['legends'][i])
        axs[idxes].scatter(pargs['x_tick_labels'], db_thresholds[:,1,i], s=pargs['markersize'], marker=pargs['markers'][i], edgecolor=pargs['colors'][i], facecolor=pargs['colors'][i], linewidths=pargs['linewidth'])
        axs[idxes].set_xlim(-0.5, n_templates-0.5)
        axs[idxes].set_ylim(db_th_min, db_th_max)
        axs[idxes].legend(loc='best', fontsize=pargs['fontsizes'][1])
        
        axs[idxes].tick_params(axis='x', which='both', direction='out', length=0, width=0,pad=5, labelsize=pargs['fontsizes'][2], labelbottom=True, labeltop=False, grid_color='k', grid_alpha=1, grid_linewidth=1, grid_linestyle='--')
        axs[idxes].grid(visible=True, which='minor', axis='x', linestyle='--', linewidth=pargs['linewidth'])
        axs[idxes].tick_params(axis='y', which='major', direction='out', length=12, width=4, pad=3, labelsize=pargs['fontsizes'][2], left=True, right=True, labelleft=True, labelright=True)
        axs[idxes].tick_params(axis='y', which='minor', direction='out', length=8, width=4, left=True, right=True, labelleft=False, labelright=False)

    fig.text(0.5, -0.05, pargs['x_label'], ha='center', fontsize=pargs['fontsizes'][0])
    fig.text(-0.05, 0.5, pargs['y_label'], va='center', rotation='vertical', fontsize=pargs['fontsizes'][0])
    savefig('plot_accuracy_' + pargs['plot_name'] + '.svg', dpi=300, bbox_inches='tight')
    close()

In [ ]:
# plot
n_mod = len(models)
n_row = 2
n_col = math.ceil(n_mod / 2)
pargs = {'plot_name': 'detection-1-f-noise', 'figsize': (20, 8), 'n_rows': n_row, 'n_cols': n_col, 'fontsizes': [36, 18, 15], 'x_label': 'Target', 'y_label': 'Threshold (dB)', 'x_tick_labels': ['rc', 'sine', 'tri', 'sqr'], 'legends': ['TM', 'WTM', 'RTM', 'WRTM'], 'markers': ['D', 'D', 'D', 'D'], 'colors': ['steelblue', 'firebrick', 'forestgreen', 'indigo'], 'markersize': 500, 'linewidth': 3}
plot_double_bin_dp(spat_sim_all, amp_sim_all, dp_all, pargs)